In [ ]:
!pip install diffusers transformers torch torchvision accelerate safetensors huggingface_hub --quiet

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
pipe.to("cuda")

prompt = "a helicopter, dslr, ultra quality, sharp focus, tack sharp, dof, film grain, Fujifilm XT3, crystal clear, 8K UHD, highly detailed"
neg_prompt = "((text:1.9)), ((title:1.9)), ((lettering:1.9)), ((cropped:1.9)), ((words:1.9)), 3d, (worst quality:1.9), (low quality:1.9), (normal quality:1.9), (bad-hands-5:1.9), lowres, out of focus, hazy, oversaturated, signature, (watermark:1.9), username, artist name, error, bad image, bad photo, ng_deepnegative_v1_75t"

image = pipe(prompt=prompt, neg_prompt=neg_prompt, num_inference_steps=4, strength=0.5, guidance_scale=0.0).images[0]
image.save("image4.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# Pixar Cars

In [ ]:
!git clone https://github.com/replicate/cog-sdxl cog_sdxl

In [ ]:
import torch
from huggingface_hub import hf_hub_download
from diffusers import DiffusionPipeline
from cog_sdxl.dataset_and_utils import TokenEmbeddingsHandler
from diffusers.models import AutoencoderKL

pipe = DiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
).to("cuda")

pipe.load_lora_weights("fofr/sdxl-pixar-cars", weight_name="lora.safetensors")

text_encoders = [pipe.text_encoder, pipe.text_encoder_2]
tokenizers = [pipe.tokenizer, pipe.tokenizer_2]

embedding_path = hf_hub_download(repo_id="fofr/sdxl-pixar-cars", filename="embeddings.pti", repo_type="model")
embhandler = TokenEmbeddingsHandler(text_encoders, tokenizers)
embhandler.load_embeddings(embedding_path)
prompt="A photo of a blue <s0><s1> Pixar car with eyes, photography, 50mm"
images = pipe(
    prompt,
    cross_attention_kwargs={"scale": 0.8},
).images
#your output image
images[0]
image.save("output-pixar.png")


Free GPU Memory

In [33]:
torch.cuda.empty_cache()

In [32]:
 !sudo nvidia-smi --gpu-reset -i 0

NVIDIA-SMI couldn't find libnvidia-ml.so library in your system. Please make sure that the NVIDIA Display Driver is properly installed and present in your system.
Please also try adding directory that contains libnvidia-ml.so to your system PATH.
